
### Step 1 - Create Training Data

In [ ]:
import cv2
import numpy as np

# Load HAAR face classifier
face_classifier = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

# Load functions
def face_extractor(img):
    # Function detects faces and returns the cropped face
    # If no face detected, it returns the input image
    
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    faces = face_classifier.detectMultiScale(gray, 1.3, 5)
    
    if faces is ():
        return None
    
    # Crop all faces found
    for (x,y,w,h) in faces:
        cropped_face = img[y:y+h, x:x+w]

    return cropped_face

# Initialize Webcam
cap = cv2.VideoCapture(0)
count = 0

# Collect 100 samples of your face from webcam input
while True:

    ret, frame = cap.read()
    if face_extractor(frame) is not None:
        count += 1
        face = cv2.resize(face_extractor(frame), (200, 200))
        face = cv2.cvtColor(face, cv2.COLOR_BGR2GRAY)

        # Save file in specified directory with unique name
        file_name_path = 'D:/LW/Tasks/TASK 6/face1/' +'face'+str(count) + '.jpg'
        #print(file_name_path)
        cv2.imwrite(file_name_path, face)

        # Put count on images and display live count
        cv2.putText(face, str(count), (50, 50), cv2.FONT_HERSHEY_COMPLEX, 1, (0,255,0), 2)
        cv2.imshow('Face Cropper', face)
        
    else:
        print("Face not found")
        pass

    if cv2.waitKey(1) == 13 or count == 100: #13 is the Enter Key
        break
        
cap.release()
cv2.destroyAllWindows()      
print("Collecting Samples Complete")

### Step 2 - Train Model

In [ ]:
cap.release()

In [ ]:
import cv2
import numpy as np
from os import listdir
from os.path import isfile, join

# Get the training data we previously made
data_path = 'D:/LW/Tasks/TASK 6/face1/'
onlyfiles = [f for f in listdir(data_path) if isfile(join(data_path, f))]

# Create arrays for training data and labels
Training_Data, Labels = [], []

# Open training images in our datapath
# Create a numpy array for training data
for i, files in enumerate(onlyfiles):
    image_path = data_path+onlyfiles[i]
    images = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    Training_Data.append(np.array(images,'uint8'))
    Labels.append(i)

# Create a numpy array for both training data and labels
Labels = np.asarray(Labels, dtype=np.int32)

# Initialize facial recognizer
#model = cv2.face.createLBPHFaceRecognizer()
# NOTE: For OpenCV 3.0 use cv2.face.createLBPHFaceRecognizer()
# pip install opencv-contrib-python
# model = cv2.createLBPHFaceRecognizer()

#vimal_model  = cv2.face_LBPHFaceRecognizer.create()
model=cv2.face.LBPHFaceRecognizer_create()
# Let's train our model 
model.train(Training_Data,Labels)
print("Model trained successfully")

In [ ]:
#to send email when your face is detected
import smtplib

def email():
    sender = 'xxx.yyy@gmail.com'
    receivers ='xxx.yyy@gmail.com'

    message = 'hi divija your face is detected'


    s = smtplib.SMTP(host='smtp.gmail.com', port=587)
    s.starttls()
    s.login('xxx.yyy@gmail.com','xxxx')
    s.sendmail(sender, receivers,message)
    print('email sent successfully')

In [ ]:
#to send whatsapp message when your face is detected
import pywhatkit as pw
def whatsapp_msg():
    pw.sendwhatmsg_instantly("+91{xxxxxxxxxx}","hi divija")
    print('whatsapp message sent successfully')

In [ ]:
# create a new EC2 instance

import boto3
ec2 = boto3.resource('ec2')

def launch_instance():
    instances = ec2.create_instances(
         ImageId='ami-0ad704c126371a549',
         MinCount=1,
         MaxCount=1,
         InstanceType='t2.micro',
         KeyName='keyec2'
     )
    print("successfully created instance")

In [ ]:
#create new volume
def create_volume(ap):
    ebs_vol = ec2.create_volume(
            Size=5,
            AvailabilityZone=ap
        )
    print("successfully created volume")

In [ ]:
def stop_instance():
    print("enter instance id")
    instance_id=input()
    os.system("aws ec2 stop-instances --instance-ids "+instance_id+"")
    print("successfully stopped instance")

In [ ]:
import boto3
client = boto3.client('ec2')
def attach_volume():
    print("enter instance id")
    instance_id=input()
    print("enter volume id")
    volume_id=input()
    result = client.attach_volume(VolumeId=volume_id,InstanceId=instance_id,Device='/dev/xvdi')
    print("successfully attached volume")

### Step 3 - Run Our Facial Recognition

In [ ]:
import cv2
import numpy as np
import os


face_classifier = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

def face_detector(img, size=0.5):
    
    # Convert image to grayscale
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    faces = face_classifier.detectMultiScale(gray, 1.3, 5)
    if faces is ():
        return img, []
    
    
    for (x,y,w,h) in faces:
        cv2.rectangle(img,(x,y),(x+w,y+h),(0,255,255),2)
        roi = img[y:y+h, x:x+w]
        roi = cv2.resize(roi, (200, 200))
    return img, roi

In [ ]:
def display_image(image):
    cv2.imshow('Face Recognition', image )
    cv2.waitKey()
    cv2.destroyAllWindows()

In [ ]:
# Open Webcam
#cap = cv2.VideoCapture(0)
#while True:
    #ret, frame = cap.read()
photo=input("enter the location of the image")
frame=cv2.imread(photo)
display_image(frame)
image, face = face_detector(frame)
try:
    face = cv2.cvtColor(face, cv2.COLOR_BGR2GRAY)
    # "results" comprises of a tuple containing the label and the confidence value
    results = model.predict(face)
    if results[1] < 500:
        confidence = int( 100 * (1 - (results[1])/400) )
        display_string = str(confidence) + '% Confident it is User'
        
    cv2.putText(image, display_string, (100, 120), cv2.FONT_HERSHEY_COMPLEX, 1, (255,120,150), 2)
    if confidence > 80:
        cv2.putText(image, "Hi Divija", (250, 450), cv2.FONT_HERSHEY_COMPLEX, 1, (0,255,0), 2)
        display_image(image)
        email()
        whatsapp_msg()
    else:
        cv2.putText(image, "I dont know, how r u", (250, 450), cv2.FONT_HERSHEY_COMPLEX, 1, (0,0,255), 2)
        display_image(image)
        launch_instance()
        ap=input("enter availability zone")
        create_volume(ap)
        stop_instance()
        attach_volume()
except:
        cv2.putText(image, "No Face Found", (220, 120) , cv2.FONT_HERSHEY_COMPLEX, 1, (0,0,255), 2)
        cv2.putText(image, "looking for face", (250, 450), cv2.FONT_HERSHEY_COMPLEX, 1, (0,0,255), 2)
        display_image(image)
        pass

In [ ]:
# Open Webcam
#cap = cv2.VideoCapture(0)
#while True:
    #ret, frame = cap.read()
photo=input("enter the location of the image")
frame=cv2.imread(photo)
display_image(frame)
image, face = face_detector(frame)
try:
    face = cv2.cvtColor(face, cv2.COLOR_BGR2GRAY)
    # "results" comprises of a tuple containing the label and the confidence value
    results = model.predict(face)
    if results[1] < 500:
        confidence = int( 100 * (1 - (results[1])/400) )
        display_string = str(confidence) + '% Confident it is User'
        
    cv2.putText(image, display_string, (100, 120), cv2.FONT_HERSHEY_COMPLEX, 1, (255,120,150), 2)
    if confidence > 80:
        cv2.putText(image, "Hi Divija", (250, 450), cv2.FONT_HERSHEY_COMPLEX, 1, (0,255,0), 2)
        display_image(image)
        email()
        whatsapp_msg()
    else:
        cv2.putText(image, "I dont know, how r u", (250, 450), cv2.FONT_HERSHEY_COMPLEX, 1, (0,0,255), 2)
        display_image(image)
        launch_instance()
        ap=input("enter availability zone")
        create_volume(ap)
        stop_instance()
        attach_volume()
except:
        cv2.putText(image, "No Face Found", (220, 120) , cv2.FONT_HERSHEY_COMPLEX, 1, (0,0,255), 2)
        cv2.putText(image, "looking for face", (250, 450), cv2.FONT_HERSHEY_COMPLEX, 1, (0,0,255), 2)
        display_image(image)
        pass

In [ ]:
#cap.release()

In [ ]:
# D:/LW/Tasks/TASK 6/divija_photo.jpg

In [ ]:
# D:/LW/Tasks/TASK 6/teja_photo.jpg